In [ ]:
from IPython.display import display
import gc
import joblib as jb
from tqdm.auto import tqdm, trange
import os

# data manipulation
import numpy as np
import pandas as pd
import sqlite3 as sq
from sklearn import preprocessing, pipeline

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# unsupervised learning
from sklearn import decomposition, cluster

# Introduction

This notebook aims to clusterize motorcycles' collisions entries and confront them with motorcycle make information.

First `collisions` and `parties` tables are read excluding descriptive features.
Then they are preprocessed filling missing values and encoding categorical features.
Eventually different tables' information are merged together to build a single table.

# Data preprocessing

In [ ]:
con = sq.connect("../input/california-traffic-collision-data-from-switrs/switrs.sqlite")
cur = con.cursor()

In [ ]:
tables = [name[0] for name in cur.execute("SELECT name FROM sqlite_master")]
print("Tables:")
print(tables)

For the purpose of this notebook only `collisions` and `parties` tables are used.

In [ ]:
tables = ["collisions", "parties"]
print("Used tables: ")
print(tables)

In [ ]:
cols = {table: list(map(lambda x: x[0], cur.execute("SELECT * FROM {}".format(table)).description)) for table in tables}
print("Tables' features:")
print()
for key,val in cols.items():
    print("="*5 + " " + key + " " + "="*5)
    display(val)
    print()

Not all columns are relevant to characterize motorcycle collisions.

In [ ]:
# all features are copied in this dictionary; commented ones are removed because they are only descriptive for the target of this work

cols = {
    "collisions":
    [
        "case_id",
        #"jurisdiction",
        #"officer_id",
        #"reporting_district",
        #"chp_shift",
        #"population",
        "county_city_location",
        "county_location",
        "special_condition",
        "beat_type",
        "chp_beat_type",
        "city_division_lapd",
        "chp_beat_class",
        "beat_number",
        "primary_road",
        "secondary_road",
        "distance",
        "direction",
        "intersection",
        "weather_1",
        "weather_2",
        "state_highway_indicator",
        "caltrans_county",
        "caltrans_district",
        "state_route",
        "route_suffix",
        "postmile_prefix",
        "postmile",
        "location_type",
        "ramp_intersection",
        "side_of_highway",
        "tow_away",
        "collision_severity",
        "killed_victims",
        "injured_victims",
        "party_count",
        "primary_collision_factor",
        "pcf_violation_code",
        "pcf_violation_category",
        "pcf_violation",
        "pcf_violation_subsection",
        "hit_and_run",
        "type_of_collision",
        "motor_vehicle_involved_with",
        "pedestrian_action",
        "road_surface",
        "road_condition_1",
        "road_condition_2",
        "lighting",
        "control_device",
        "chp_road_type",
        "pedestrian_collision",
        "bicycle_collision",
        "motorcycle_collision",
        "truck_collision",
        "not_private_property",
        "alcohol_involved",
        "statewide_vehicle_type_at_fault",
        "chp_vehicle_type_at_fault",
        "severe_injury_count",
        "other_visible_injury_count",
        "complaint_of_pain_injury_count",
        "pedestrian_killed_count",
        "pedestrian_injured_count",
        "bicyclist_killed_count",
        "bicyclist_injured_count",
        "motorcyclist_killed_count",
        "motorcyclist_injured_count",
        "primary_ramp",
        "secondary_ramp",
        "latitude",
        "longitude",
        "collision_date",
        "collision_time",
        #"process_date"
    ],
    
    "parties":
    [
        #"id",
        "case_id",
        "party_number",
        "party_type",
        "at_fault",
        "party_sex",
        "party_age",
        "party_sobriety",
        "party_drug_physical",
        "direction_of_travel",
        "party_safety_equipment_1",
        "party_safety_equipment_2",
        "financial_responsibility",
        "hazardous_materials",
        "cellphone_in_use",
        "cellphone_use_type",
        "school_bus_related",
        "oaf_violation_code",
        "oaf_violation_category",
        "oaf_violation_section",
        "oaf_violation_suffix",
        "other_associate_factor_1",
        "other_associate_factor_2",
        "party_number_killed",
        "party_number_injured",
        "movement_preceding_collision",
        "vehicle_year",
        "vehicle_make",
        #"statewide_vehicle_type",
        #"chp_vehicle_type_towing",
        #"chp_vehicle_type_towed",
        "party_race"
    ]
}

print("Used features: ")
print()
for key,val in cols.items():
    print("="*5 + " " + key + " " + "="*5)
    display(val)
    print()

Now `collisions` table is read.

In [ ]:
def get_str_list(L):
    S = ""
    for x in L:
        S += x + ", "
    S = S[:-2]
    return S

df = {}

# get collisions with a mototrcycle involved
cmd = "SELECT " + get_str_list(cols["collisions"]) + " FROM collisions WHERE motorcycle_collision=1"
df["collisions"] = pd.DataFrame([list(x) for x in cur.execute(cmd)], columns=cols["collisions"])

display(df["collisions"].head())
print("\nEntries: {}".format(df["collisions"].shape[0]))
print("Features: {}".format(df["collisions"].shape[1]))

Timestamps are splitted in date and time and they are not parsed, so they are processed generating one column per field (i.s. year, month, day, hour, minute, second).
The columns to consider are `collision_date` and `collision_time`.

In [ ]:
df["collisions"].loc[:, ["collision_date", "collision_time"]].isnull().sum()

`collision_time` has some missing values.
Median value of timestamps distribution within the 24-hours day is used to fill missing values.

In [ ]:
def get_seconds(hour, minute, second):
    return hour*3600 + minute*60 + second

def get_hms(seconds):
    hour = seconds // 3600
    seconds -= hour * 3600
    minute = seconds // 60
    seconds -= minute * 60
    second = round(seconds)
    return int(hour), int(minute), int(second)

def get_date_cols(col_date, col_time=None):
    
    year = col_date.apply(lambda x: pd.Timestamp(x).year)
    month = col_date.apply(lambda x: pd.Timestamp(x).month)
    day = col_date.apply(lambda x: pd.Timestamp(x).day)
    
    df = pd.DataFrame({"year": year.values, "month": month.values, "day": day.values}, index=col_date.index)
    
    if type(col_time) == pd.Series:
        
        # fill missing times with median
        time_median = col_time.apply(lambda x: get_seconds(*np.array(x.split(":")).astype(int)) if type(x)==str else np.nan).dropna().median()
        col_time = col_time.fillna(value="{:02d}:{:02d}:{:02d}".format(*get_hms(time_median)))

        hour = col_time.apply(lambda x: x.split(":")[0]).astype(int)
        minute = col_time.apply(lambda x: x.split(":")[1]).astype(int)
        second = col_time.apply(lambda x: x.split(":")[2]).astype(int)
        
        # pack time fields with date's
        df = df.join(pd.DataFrame({"hour": hour, "minute": minute, "second": second}, index=col_time.index))
    
    return df

df["collisions"] = df["collisions"].join(get_date_cols(df["collisions"].collision_date, df["collisions"].collision_time).rename(columns={var: "collision_date_"+var for var in ["year", "month", "day", "hour", "minute", "second"]}))
df["collisions"] = df["collisions"].drop(columns=["collision_date", "collision_time"])

df["collisions"].loc[:, ["collision_date_" + var for var in ["year", "month", "day", "hour", "minute", "second"]]].describe()

Now missing values are filled using `"None"` label for categorical and $0$ for numeric features.
Maybe $0$ is not the best value to use but it does not belongs to other significant values of numeric columns.

In [ ]:
def preprocess_missing(df, figsize=(30,5), none_is_na = True):
    
    ''' Fill missing values: with "None" for labels, with 0 for numeric features '''
    
    df = df.copy()
    
    if none_is_na:
        df = df.replace("None", np.nan)
    
    null_counts = (df.isnull().sum() / df.shape[0] * 100).loc[lambda x: x!=0]
    
    fig,ax = plt.subplots(figsize=figsize)
    ax = sns.barplot(x=null_counts.index, y=null_counts.values, color="C0", ax=ax)
    ax.tick_params(axis="x", rotation=45)
    ax.set_title("Missing values")
    ax.set_xlabel("feature")
    ax.set_ylabel("percentage (%)")
    plt.show()
    
    drop_cols = null_counts.loc[lambda x: x==100].index.tolist()
    df = df.drop(columns=drop_cols)
    if len(drop_cols) != 0:
        print("Completely null features:")
        for col in drop_cols:
            print("\t"+col)
    
    null_obj = df.dtypes.loc[null_counts.index].loc[lambda x: x==object]
    null_numeric = df.dtypes.loc[null_counts.index].loc[lambda x: x!=object]
    
    df.loc[:,null_obj.index] = df.loc[:,null_obj.index].fillna(value="None")
    
    display(df.loc[:,null_numeric.index].describe())
    
    # all numeric features with missing values can be filled with 0 in all tables, because of this the following line of code is commented
    #fill_numeric = input("Fill missing numeric values with 0? (Y/n): ")
    fill_numeric = "y"
    
    if fill_numeric in ["Y", "y", ""]:
        df.loc[:, null_numeric.index] = df.loc[:, null_numeric.index].fillna(value=0)
        
    return df

df["collisions"] = preprocess_missing(df["collisions"])

Now `parties` table is read selecting only entries associated with a motorcycle.

In [ ]:
# get collisions' parties on a motorcycle (or scooter)
cmd = "SELECT " + get_str_list(cols["parties"]) + " FROM parties "
cmd += "WHERE case_id IN (" + get_str_list(df["collisions"].case_id) + ") AND statewide_vehicle_type = 'motorcycle or scooter'"
df["parties"] = pd.DataFrame([list(x) for x in cur.execute(cmd)], columns=cols["parties"])

display(df["parties"].head())
print("\nEntries: {}".format(df["parties"].shape[0]))
print("Features: {}".format(df["parties"].shape[1]))

In [ ]:
df["parties"] = preprocess_missing(df["parties"])

In [ ]:
print("Motorcycles without make: {}".format((df["parties"].vehicle_make == "None").sum()))

Motorcycles without a make are not useful so they are removed.

In [ ]:
df["parties"] = df["parties"].loc[df["parties"].vehicle_make != "None"]

Collisions data are merged into `parties` table using case ids.
Moreover, in this way collisions without any party entry will be excluded.

In [ ]:
df = df["parties"].merge(df["collisions"], on="case_id", how="left").set_index(["case_id", "party_number"]).sort_index()
print("Dataset shape: {}".format(df.shape))

Now categorical features are encoded.

In [ ]:
def encode_labels(df, progress=True):
    df = df.copy()
    label_encoders = {}
    for col in tqdm(df.dtypes.loc[lambda x: x==object].index, disable=not progress):
        label_encoders[col] = preprocessing.LabelEncoder().fit(df.loc[:,col])
        df.loc[:,col] = label_encoders[col].transform(df.loc[:,col])
    return df, label_encoders

df_enc, label_encoders = encode_labels(df)

In following learning `vehicle_make` is excluded from analysis to bring back it later when generated clusters are considered.

In [ ]:
targets = ["vehicle_make"]
x,y = df_enc.drop(columns=targets), df_enc.loc[:,targets]

In [ ]:
N = 10 # how many makes to show separately
vehicle_make_frac = (df.vehicle_make.value_counts() / df.shape[0]).sort_values(ascending=False).iloc[:N]
vehicle_make_frac = vehicle_make_frac.append(pd.Series([1 - vehicle_make_frac.sum()], index=["other"]))

fig,ax = plt.subplots(figsize=(10,5))
ax.set_title("Vehicle makes")
ax.set_ylabel("relative fraction")
ax.set_xlabel("make")
ax.tick_params(axis="x", rotation=45)
ax = sns.barplot(x=vehicle_make_frac.index, y=vehicle_make_frac.values, color="C0", ax=ax)

del vehicle_make_frac

# Principal Component Analysis

PCA could be used to reduce the dimensionality of the data.
This is possible only if a restricted number of the first principal components contain a sufficient amount of the original variance.

In [ ]:
def get_pca_pipe(x):
    ''' Fit PCA and return model '''
    pca_pipe = pipeline.Pipeline([
        ("scaler", preprocessing.StandardScaler()),
        ("pca", decomposition.PCA(random_state=1))
    ])
    pca_pipe = pca_pipe.fit(x)
    return pca_pipe

pca_pipe = get_pca_pipe(x)
pca = pca_pipe.named_steps["pca"]

ax = sns.lineplot(x=np.arange(1,pca.n_components_+1), y=pca.explained_variance_ratio_.cumsum())
ax.set_title("Train set PCA")
ax.set_xlabel("principal components")
_ = ax.set_ylabel("cum. explained var. ratio")

First principal components do not contain much of the original variance to be representative of the starting space, so PCA results are not used.

# Clustering

Before introducing clusters in data, their inertia is studied varying the groups number used by estimator.
Then clusters are produced using a value near the elbow of the resulting inertia plot.

In [ ]:
def get_cluster_pipe(x, n, batch_size=1000):
    cluster_pipe = pipeline.Pipeline([
        ("scaler", preprocessing.StandardScaler()),
        ("cluster", cluster.MiniBatchKMeans(n_clusters=n, batch_size=batch_size, random_state=1))
    ])
    cluster_pipe = cluster_pipe.fit(x)
    return cluster_pipe

N = 150 # maximum clusters tested
inertia = [cluster_pipe.named_steps["cluster"].inertia_ for cluster_pipe in jb.Parallel(n_jobs=-1)(jb.delayed(get_cluster_pipe)(x, n) for n in trange(2,N+1))]

ax = sns.lineplot(x=np.arange(2,N+1), y=inertia)
ax.set_title("Train set clustering inertia")
ax.set_xlabel("n_clusters")
_ = ax.set_ylabel("inertia")

$60$ clusters are used.

In [ ]:
cluster_pipe = get_cluster_pipe(x, n=60)
cluster_values = cluster_pipe.predict(x)
x.insert(loc=x.shape[1], column="cluster", value=cluster_values)
df.insert(loc=df.shape[1], column="cluster", value=cluster_values)
df_enc.insert(loc=df_enc.shape[1], column="cluster", value=cluster_values)

Clusters' properties can be retrieved form their centers.

In [ ]:
def decode_labels(df, encoders, progress=False):
    df = df.copy()
    for col in encoders.keys():
        if col in df.columns:
            df.loc[:,col] = encoders[col].inverse_transform(df.loc[:,col].apply(lambda x: round(x))) # decode with the nearest valid label
    return df

# cluster characteristics
cluster_centers = pd.DataFrame(cluster_pipe.named_steps["scaler"].inverse_transform(cluster_pipe.named_steps["cluster"].cluster_centers_), columns=x.drop(columns="cluster").columns)
cluster_centers.index.name = "cluster"
cluster_centers = decode_labels(cluster_centers, label_encoders)
cluster_centers.head()

Since clustering did not take into account `vehicle_make`, motorcycles could belong to one or more clusters based on collisions characteristics.
Because of this next table shows the number of clusters that can be counted counted for each make.

In [ ]:
# clusters counts per vehicle make
cluster_counts = pd.DataFrame({"clusters": df.groupby("vehicle_make").cluster.nunique()})
cluster_counts = cluster_counts.sort_index().sort_values("clusters", ascending=False)
cluster_counts.head()

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
ax.set_title("Makes' clusters")
ax.set_xlabel("Make's number of clusters")
ax.set_ylabel("relative fraction")
ax = sns.histplot(data=cluster_counts, x="clusters", stat="probability", bins=np.arange(1,cluster_pipe.named_steps["cluster"].cluster_centers_.shape[0] + 2) - 0.5, ax=ax)

More than $60\%$ of makes has collisions belonging only to one cluster.
This means that those motorcycles made incidents with approximately the same characteristics, attributable to a single group.

In [ ]:
# (vehicle_make, cluster) collisions counts
collision_counts = pd.DataFrame({"collisions": df.loc[:, ["vehicle_make", "cluster"]].value_counts()})
collision_counts = collision_counts.sort_values("collisions", ascending=False)
collision_counts.head()

In **Data preprocessing** section an histogram showing the relative fraction of collisions per make is already present.
Now the following plot shows the relative fraction of collisions per cluster.

In [ ]:
collision_counts_cluster = collision_counts.groupby("cluster").collisions.sum() / collision_counts.collisions.sum()

fig,ax = plt.subplots(figsize=(15,5))
ax.set_title("Clusters collisions")
ax.set_xlabel("cluster")
ax.set_ylabel("relative fraction")
ax = sns.barplot(x=collision_counts_cluster.index, y=collision_counts_cluster.values, ax=ax, color="C0")

Now final results are saved.

In [ ]:
encoders_dir = "encoders"
os.mkdir(encoders_dir)
for col in label_encoders.keys():
    jb.dump(label_encoders[col], encoders_dir + "/" + col + ".pkl")

estimators_dir = "estimators"
os.mkdir(estimators_dir)
jb.dump(cluster_pipe, estimators_dir + "/cluster_pipe.pkl")

cluster_centers.to_csv("cluster_centers.csv")
cluster_counts.to_csv("cluster_counts.csv")
collision_counts.to_csv("collision_counts.csv")

# Conclusions

Work presented in this report shows how clustering could help determine common properties in one motorcycle make incidents.
The problem is that not all motorcycles can be assigned to a single type of collision: in fact many of the seen entries (about $40\%$) show for the same make different groups.
Althought the non-unique correspondence, $60\%$ of collisions presents one cluster per make.

Eventually, further improvements to this notebook could be training a supervised model capable of predicting motorcycle type based on collision characteristics; in this way relation between incidents properties and motorcycle make will be further proved.